QR Code Detection

In [1]:
import ipywidgets.widgets as widgets
import traitlets
import cv2
import numpy as np
import PIL.Image
import time
import datetime
from jetbot import Robot
from jetbot import Camera
from jetbot import bgr8_to_jpeg

In [2]:
print("OpenCV version:", cv2.__version__)

OpenCV version: 4.1.1


In [10]:
! python3 --version

Python 3.6.9


In [2]:
image = widgets.Image(format='jpeg', width=224, height=224)
display(image)

Image(value=b'', format='jpeg', height='224', width='224')

In [3]:
camera = Camera.instance(width=820, height=616, capture_width=3280, capture_height=2464)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

In [ ]:
camera.stop()

In [ ]:
# Label widget to display QR code data in the notebook
qr_code_display = widgets.Label(value="QR Code: None")
display(qr_code_display)

In [8]:
def detect_qr_code(image):
    timestamp = '{:%Y-%m-%d %H:%M:%S}'.format(datetime.datetime.now())
    #image = PIL.Image.fromarray(image)
    # Convert the image to a format compatible with OpenCV (assuming it's a PIL image)
    # check image type, no need to convert if numpy.ndarray
    #print(type(image))
    #print(image.shape)
    #image_cv = np.array(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #edges = cv2.Canny(gray, 100, 200)
    # Optional: Apply adaptive thresholding to increase contrast
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

    #cv2.imwrite(f'output_image_{timestamp}.jpg', image)
    #cv2.imwrite(f'thresh_image_{timestamp}.jpg', thresh)
    #cv2.imwrite(f'output_gray_{timestamp}.jpg', gray)
    #cv2.imwrite(f'output_edges_{timestamp}.jpg', edges)
    
    # Initialize the QR code detector
    qr_decoder = cv2.QRCodeDetector()
    #qr_decoder = cv2.BarcodeDetector()
    
    _, points = qr_decoder.detect(image)
    if points is not None:
        print("QR Code detected, but not decoded")
    else:
        print("No QR Code detected.")
    
    # Detect and decode the QR code
    data, points, _ = qr_decoder.detectAndDecode(image)
    if points is not None and data:
        # QR code detected and decoded
        print(f"Code detected! {data}")
        qr_code_display.value = f"QR Code: {data}"  # Return the decoded data (the content of the QR code)
    else:
        # No QR code found
        #print("No Code detected!")
        return None

In [16]:
value=0
while(value<1):
    detect_qr_code(camera.value)
    time.sleep(1)
    value = value + 1

QR Code detected, but not decoded


In [ ]:
camera.value

In [ ]:
camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing


Some place to hold stuff for copying

In [ ]:
# Global variable to hold the QR code data
qr_code_data = None

# Label widget to display QR code data in the notebook
qr_code_display = ipywidgets.Label(value="QR Code: None")
display(qr_code_display)

def detect_qr_code(image):
    # Convert the image to a format compatible with OpenCV (assuming it's a PIL image)
    image_cv = np.array(image)
    
    # Initialize the QR code detector
    qr_decoder = cv2.QRCodeDetector()
    
    # Detect and decode the QR code
    data, points, _ = qr_decoder.detectAndDecode(image_cv)
    
    if points is not None and data:
        # QR code detected and decoded
        return data  # Return the decoded data (the content of the QR code)
    else:
        # No QR code found
        return None
    
def uTurn(change):
    robot.left(0.5)
    robot.right(0.5)
    time.sleep(1)
    robot.stop()

#New code begins here
    
    # QR code detection logic
    detected_qr_code = detect_qr_code(image)  # Custom function to detect and decode QR code

    if detected_qr_code != None:
        qr_code_data = detected_qr_code  # Store the detected QR code data
        # Update the display widget with the detected QR code data
        qr_code_display.value = f"QR Code: {qr_code_data}"
        
        # Action based on QR code data
        uTurn()
        
        return detected_qr_code
    
    #End new code begins here